# 10. Ecrire des logs
## Pourquoi écrire des logs
Dès qu'une opération comporte un certain niveau de complexité ou un niveau d'incertitude, il devient indispensable d'écrire des logs. En effet, si quelque chose se passe mal, il est capital de pouvoir:
* Reprendre le processus là où il s'est interrompu
* Avoir des informations suffisantes sur l'erreur qui s'est produites si une erreur arrive.

## Problèmes courants
Un cas typique est une api qui ne répond plus pendant quelques instants. Cela peut faire planter un script. 

Parfois un record est atypique et fait planter le script. Il faut donc analyser ce cas, modifier le script pour prendre en compte ce cas particulier puis relancer le script.

## Masquer les erreurs: une mauvaise pratique
Un code qui plante n'est pas forcéement mauvais. On peut être tenté de cacher les cas particulier à coup de `try` et `except` mais sans réel traitement, cela ne viendrait qu'à cacher la poussières sous le tapis.

## Comment rédiger des logs
Dans Juyter, la solution la plus simple d'écrire des logs est de recourir simplement à `print`.

Seulement, Jupyter est très adapté pour de petits traitements. Dès que l'interactivité du Jupyter n'est plus un atout, mais une contrainte, il vaut mieux utiliser d'autres outils comme la bibliothèque logging.

## Logging

### Paramétrer les logs
```python
import logging
message_format = "%(asctime)s - %(levelname)s - %(message)s"
log_file_name = f'Resultat/log.txt'
logging.basicConfig(filename=log_file_name, format=message_format, level=logging.INFO)
```

### Afficher des alertes
Les logs sont organisés en différents niveaux.
```python
# Information
logging.info(f'msg')

# Avertissement
logging.warning(f'msg')

# Erreur
logging.error(f'msg')

# Erreur critique
logging.critical(f'msg')

```

## Exemple avec "print"
Dans cet exemple, l'api SRU de swisscovery sera utilisée https://slsp.ch/en/metadata.

In [42]:
import requests
import json
from lxml import etree
import logging

In [43]:
BASE_URL = 'https://swisscovery.slsp.ch/view/sru/41SLSP_NETWORK'

# Paramètres fixes de la requête
params = {'version': '1.2',
          'operation': 'searchRetrieve'}

In [44]:
# Ce script récuère le mms_id via l'api SRU de swisscovery
isbns = ['9780134546933', '9781484239124', '97814842391244', '1803242353']
mms_ids = []
for isbn in isbns:
    
    # Ajout du paramètre avec le requête elle-même
    params['query'] = f'alma.isbn={isbn} and alma.mms_tagSuppressed=false'
    
    # Interrogation de l'api
    r = requests.get(BASE_URL, params=params)
    
    # Parsing du xml
    xml = etree.XML(r.content)
    
    # Récupération du mms_id. A noter le nom de domaine indispensable pour parser du xml
    mms_ids.append(xml.find('.//{http://www.loc.gov/MARC21/slim}controlfield[@tag="001"]').text)

AttributeError: 'NoneType' object has no attribute 'text'

Dans ce script, il n'y a pas d'information sur quel ISBN a posé problème. On ne sait pas si le script plante dans tous les cas ou si c'est seulement un des isbns qui pose problème.

In [45]:
# Ce script récuère le mms_id via l'api SRU de swisscovery
isbns = ['9780134546933', '9781484239124', '97814842391244', '1803242353']
mms_ids = []
for isbn in isbns:
    
    # Affichage des logs
    print(f"Traitement de l'ISBN: {isbn}")
    
    # Ajout du paramètre avec le requête elle-même
    params['query'] = f'alma.isbn={isbn} and alma.mms_tagSuppressed=false'
    
    # Interrogation de l'api
    r = requests.get(BASE_URL, params=params)
    
    # Parsing du xml
    xml = etree.XML(r.content)
    
    # Récupération du mms_id. A noter le nom de domaine indispensable pour parser du xml
    mms_ids.append(xml.find('.//{http://www.loc.gov/MARC21/slim}controlfield[@tag="001"]').text)

Traitement de l'ISBN: 9780134546933
Traitement de l'ISBN: 9781484239124
Traitement de l'ISBN: 97814842391244


AttributeError: 'NoneType' object has no attribute 'text'

On voit immédiatement que c'est l'isbn "97814842391244" qui a posé problème. En fait cet ISBN est faut et la requête ne retourne donc pas de résultat. Selon l'objectif du traitement, il faut donc ensuite décider ce que l'on fait des ISBN faux ou des cas où il n'y aurait pas de résultat dans swisscovery. Les logs permettent de détecter rapidement la source du problème.

## Exemple avec "logging"
Logging est une bibliothèque permettant de rédiger relativement des logs. La bibliothèque permet d'enregistrer les logs dans un fichier. C'est donc particulièrement adapté si on lance un script en ligne de commande et qu'on ne va pas surveiller l'affichage dans la console. Cela présente plusieurs avantages:
* Les logs sont enregistrés sur le disque et ne reposent pas sur un affichage
* Sur de grandes quantités de données, les logs peuvent être analysés via des scripts, c'est parfois utile pour compter les erreurs par exemples

Documentation: https://docs.python.org/3/library/logging.html

In [46]:
# Le format est important, dans l'exemple, chaque ligne comportera la date et l'heur, le niveau et le message
message_format = "%(asctime)s - %(levelname)s - %(message)s"

# Fichier destination des logs
log_file_name = f'Resultat/log.txt'

# Configuration des logs
logging.basicConfig(filename=log_file_name, format=message_format, level=logging.INFO)



In [47]:
# Réinitialisation du fichier de log
f = open('Resultat/log.txt' ,'w')
f.close()

# Ce script récuère le mms_id via l'api SRU de swisscovery
isbns = ['9780134546933', '9781484239124', '97814842391244', '1803242353']
mms_ids = []
for isbn in isbns:
    
    # Affichage des logs
    logging.info(f"Traitement de l'ISBN: {isbn}")
    
    # Ajout du paramètre avec le requête elle-même
    params['query'] = f'alma.isbn={isbn} and alma.mms_tagSuppressed=false'
    
    # Interrogation de l'api
    r = requests.get(BASE_URL, params=params)
    
    # Parsing du xml
    xml = etree.XML(r.content)
    
    # Récupération du mms_id. A noter le nom de domaine indispensable pour parser du xml
    controlfield001 = xml.find('.//{http://www.loc.gov/MARC21/slim}controlfield[@tag="001"]')
    if controlfield001 is None:
        # Ecriture d'un avertissment dans les logs
        logging.warning(f'ISBN: {isbn} -> not notice found.')
        
        # Passage au record suivant
        continue
    mms_id = xml.find('.//{http://www.loc.gov/MARC21/slim}controlfield[@tag="001"]').text     
    mms_ids.append(mms_id)
    
    # Ecriture d'une info pour confirmer la récupération du MMS ID
    logging.info(f"ISBN: {isbn} -> MMS ID trouvé: {mms_id}")

In [48]:
# Affichage des logs
with open('Resultat/log.txt') as f:
    txt = f.read()

print(txt)

2022-04-16 19:11:21,160 - INFO - Traitement de l'ISBN: 9780134546933
2022-04-16 19:11:21,415 - INFO - ISBN: 9780134546933 -> MMS ID trouvé: 991128258669705501
2022-04-16 19:11:21,416 - INFO - Traitement de l'ISBN: 9781484239124
2022-04-16 19:11:21,708 - INFO - ISBN: 9781484239124 -> MMS ID trouvé: 991170322429705501
2022-04-16 19:11:21,708 - INFO - Traitement de l'ISBN: 97814842391244
2022-04-16 19:11:21,877 - WARNING - ISBN: 97814842391244 -> not notice found.
2022-04-16 19:11:21,878 - INFO - Traitement de l'ISBN: 1803242353
2022-04-16 19:11:22,273 - INFO - ISBN: 1803242353 -> MMS ID trouvé: 991170851082005501

